# Comparació de dos discursos de Cristóbal Montoro i Pedro Sánchez al "Debate de los Presupuestos Generales del Estado 2015, (Madrid, 22/10/2014)"

Prenent com a model aquest <a href="http://www.nytimes.com/interactive/2012/09/06/us/politics/convention-word-counts.html?_r=0">artícle</a> on s'analitzen els discursos de demòcrates i republicans a les eleccions dels EUA de 2012 intentem fer un primer projecte que serveix com a exercici per establir les passes per fer un projecte similar en el context polític espanyol.

En aquesta seguna versió es comparen els discursos de dues opcions polítiques diferents.

## Lectura de fitxers en format pdf

Mètode per extreure el texte d'un fitxer en format pdf.

(És necessari instal.lar PDFMiner ( http://www.unixuser.org/~euske/python/pdfminer/ ). Es pot instal.lar desde la linea de comandes executant: pip install pdfminer )

In [1]:
# http://stackoverflow.com/questions/26494211/extracting-text-from-a-pdf-file-using-pdfminer-in-python

from pdfminer.pdfinterp import PDFResourceManager, PDFPageInterpreter
from pdfminer.converter import TextConverter
from pdfminer.layout import LAParams
from pdfminer.pdfpage import PDFPage
from cStringIO import StringIO

def convert_pdf_to_txt(path):
    rsrcmgr = PDFResourceManager()
    retstr = StringIO()
    codec = 'utf-8'
    laparams = LAParams()
    device = TextConverter(rsrcmgr, retstr, codec=codec, laparams=laparams)
    fp = file(path, 'rb')
    interpreter = PDFPageInterpreter(rsrcmgr, device)
    password = ""
    maxpages = 0
    caching = True
    pagenos=set()

    for page in PDFPage.get_pages(fp, pagenos, maxpages=maxpages, password=password,caching=caching, check_extractable=True):
        interpreter.process_page(page)

    text = retstr.getvalue()

    fp.close()
    device.close()
    retstr.close()
    
    # http://stackoverflow.com/questions/25315566/unicodedecodeerror-in-nltks-word-tokenize-despite-i-forced-the-encoding
    text = text.decode('utf-8').lower()
    return text

#### Extracció del textes dels discursos

"Discurso de Cristóbal Montoro en el Debate de los Presupuestos Generales del Estado 2015, (Madrid, 22/10/2014)"
(descarregat de la web del Congreso: http://www.congreso.es/public_oficiales/L10/CONG/DS/PL/DSCD-10-PL-232.PDF ).

"Discurso de Pedro Sánchez en el Debate de los Presupuestos Generales del Estado 2015, (Madrid, 22/10/2014)"
(descarregat de la web de PSOE: http://www.psoe.es/source-media/000000607500/000000607955.pdf ).

In [2]:
text_pp = convert_pdf_to_txt("../data/pp/DSCD-10-PL-232_[3-11].pdf")
text_psoe = convert_pdf_to_txt("../data/psoe/000000607955.pdf")

## Anàlisi del text dels discursos

### Mètodes per netejar el texte: eliminar paraules i signes que no són rellevants.

In [3]:
import re
import string
from nltk.tokenize import word_tokenize

# Mètode per eliminar signes de puntuació
def tokenized_doc_no_punctuation(tokenized_doc):
    
    regex = re.compile('[%s]' % re.escape(string.punctuation)) 

    tokenized_words_no_punctuation = []

    for token in tokenized_doc: 
        new_token = regex.sub(u'', token)
        if not new_token == u'':
            tokenized_words_no_punctuation.append(new_token)

    return tokenized_words_no_punctuation

# Mètode per eliminar xifres numèriques
def tokenized_doc_no_numbers(tokenized_doc):
    
    regex = re.compile(re.compile(r'[()|0-9]')) 

    tokenized_words_no_numbers = []

    for token in tokenized_doc: 
        new_token = regex.sub(u'', token)
        if not new_token == u'':
            tokenized_words_no_numbers.append(new_token)

    return tokenized_words_no_numbers

In [4]:
# Conjunt de paraules (tokens) del text
tokenized_words_pp = word_tokenize(text_pp)
tokenized_words_psoe = word_tokenize(text_psoe)

In [5]:
# Eliminar els signes de puntuació i les xifres numèriques
tokenized_words_no_punctuation_pp = tokenized_doc_no_punctuation(tokenized_words_pp)
tokenized_words_no_numbers_pp = tokenized_doc_no_numbers(tokenized_words_no_punctuation_pp)

tokenized_words_no_punctuation_psoe = tokenized_doc_no_punctuation(tokenized_words_psoe)
tokenized_words_no_numbers_psoe = tokenized_doc_no_numbers(tokenized_words_no_punctuation_psoe)

### Eliminació de paraules "comunes"

- Stopwords en espanyol

- El documents utilizats contenen capçaleres i peus de pàgina (a cada pàgina) amb el següent textes:

        "DIARIO DE SESIONES DEL CONGRESO DE LOS DIPUTADOS PLENO Y DIPUTACIÓN PERMANENTE
        Núm. 232 21 de octubre de 2014"

        "OFICINA DE PRENSA FEDERAL
        C/ Ferraz, 70. 28008 Madrid. Teléfonos: 91 582 04 52 / 91 582 03 94. Fax: 91 582 04 22.
        Correo: ofiprensa@psoe.es /// www.psoe.es /// www.psoetv.es /// conferenciapolitica.psoe.es"

  TODO: Com detectar paraules o paràgrafs com aquest en els textes i eliminar aquestes repeticions?

In [6]:
from nltk.corpus import stopwords

# Mètode per eliminar les paraules que pertanyen a un conjunt de paraules donat (custom_set_of_words) 
def remove_custom_set_of_words(tokenized_doc, custom_set_of_words):
    return [ word for word in tokenized_doc if word not in custom_set_of_words ]

#### Eliminació de stopwords en espanyol

In [7]:
# Conjunt de paraules stopwords de l'espanyol del paquet nltk
print stopwords.words('spanish')

[u'de', u'la', u'que', u'el', u'en', u'y', u'a', u'los', u'del', u'se', u'las', u'por', u'un', u'para', u'con', u'no', u'una', u'su', u'al', u'lo', u'como', u'm\xe1s', u'pero', u'sus', u'le', u'ya', u'o', u'este', u's\xed', u'porque', u'esta', u'entre', u'cuando', u'muy', u'sin', u'sobre', u'tambi\xe9n', u'me', u'hasta', u'hay', u'donde', u'quien', u'desde', u'todo', u'nos', u'durante', u'todos', u'uno', u'les', u'ni', u'contra', u'otros', u'ese', u'eso', u'ante', u'ellos', u'e', u'esto', u'm\xed', u'antes', u'algunos', u'qu\xe9', u'unos', u'yo', u'otro', u'otras', u'otra', u'\xe9l', u'tanto', u'esa', u'estos', u'mucho', u'quienes', u'nada', u'muchos', u'cual', u'poco', u'ella', u'estar', u'estas', u'algunas', u'algo', u'nosotros', u'mi', u'mis', u't\xfa', u'te', u'ti', u'tu', u'tus', u'ellas', u'nosotras', u'vosostros', u'vosostras', u'os', u'm\xedo', u'm\xeda', u'm\xedos', u'm\xedas', u'tuyo', u'tuya', u'tuyos', u'tuyas', u'suyo', u'suya', u'suyos', u'suyas', u'nuestro', u'nuestra', 

In [8]:
tokenized_words_no_stopwords_pp = remove_custom_set_of_words(tokenized_words_no_numbers_pp, stopwords.words('spanish'))
tokenized_words_no_stopwords_psoe = remove_custom_set_of_words(tokenized_words_no_numbers_psoe, stopwords.words('spanish'))

#### Eliminació d'altres paraules no significatives

Aquesta part és un exemple simple (handmade) d'eliminació d'algunes paraules no significatives.
Veient els resultats posteriors em sorgeix la necessitat (i el dubte) de trobar un procediment o eines com la librería nltk per eliminar paraules (i potser expressions) com: "si" (conjunció), "toda", "sido", ...

In [9]:
# Conjunt de paraules del text del PP:
# custom_set_of_words = ['diario', 'sesiones', 'congreso', 'diputados', 'pleno', 'diputación', 'permanente', 'c', 'd']
custom_set_of_words_pp = ['diario', 'sesiones', 'congreso', 'diputados', 'pleno', u'diputaci\xf3n', 'permanente', 'c', 'd']
tokenized_words_final_pp = remove_custom_set_of_words(tokenized_words_no_stopwords_pp, custom_set_of_words_pp)

# Conjunt de paraules del text del PSOE:
# custom_set_of_words = ['oficina', 'prensa', 'federal', 'ferraz', 'madrid', 'teléfonos', 'fax', 'correo', 'm', 'c', 'w', 'psoetves', 'inform', 'ación', 'conferenciapoliticapsoees', 'psoees', 'adrid', 'ofiprensa', 'si', 'va', 'ello']
custom_set_of_words_psoe = ['oficina', 'prensa', 'federal', 'ferraz', 'madrid', u'tel\xe9fonos', 'fax', 'correo','m', 'c', 'w', 'psoetves', 'inform', u'aci\xf3n', 'conferenciapoliticapsoees', 'psoees', 'adrid', 'ofiprensa', 'si', 'va', 'ello']
tokenized_words_final_psoe = remove_custom_set_of_words(tokenized_words_no_stopwords_psoe, custom_set_of_words_psoe)

## Freqüències de les paraules més significatives

In [10]:
from collections import Counter

# Mètode que retorna un Counter amb la freqüència de les paraules d'un array de paraules
def word_freq( tokenized_doc ):
    
    tf = Counter()
    for word in tokenized_doc:
        tf[word] +=1
    return tf

In [11]:
wordCounter_pp = word_freq(tokenized_words_final_pp)
wordCounter_psoe = word_freq(tokenized_words_final_psoe)

#### Seleccionem les 20 paraules amb més ocurrències en els discursos

In [12]:
diccionari_final_pp = wordCounter_pp.most_common(20)
print "diccionari_final_pp:"
print diccionari_final_pp

diccionari_final_psoe = wordCounter_psoe.most_common(20)
print "\ndiccionari_final_psoe:"
print diccionari_final_psoe

diccionari_final_pp:
[(u'presupuestos', 60), (u'a\xf1o', 43), (u'espa\xf1a', 38), (u'gobierno', 32), (u'empleo', 32), (u'crecimiento', 30), (u'a\xf1os', 29), (u'econ\xf3mica', 28), (u'p\xfablica', 24), (u'pa\xeds', 23), (u'millones', 23), (u'crisis', 21), (u'econ\xf3mico', 21), (u'pol\xedtica', 20), (u'ahora', 19), (u'creaci\xf3n', 18), (u'p\xfablicas', 18), (u'euros', 17), (u'hoy', 16), (u'hacer', 16)]

diccionari_final_psoe:
[(u'se\xf1or', 53), (u'ustedes', 45), (u'montoro', 41), (u'pol\xedtica', 28), (u'usted', 23), (u'espa\xf1a', 23), (u'gobierno', 20), (u'pa\xeds', 17), (u'a\xf1os', 16), (u'econ\xf3mica', 14), (u'millones', 14), (u'problema', 14), (u'presupuestos', 12), (u'rajoy', 12), (u'desigualdad', 10), (u'social', 10), (u'crisis', 10), (u'familias', 10), (u'fiscal', 10), (u'menos', 9)]


In [13]:
print "Llistat de les paraules amb més ocurrències al discurs del PP:\n"
for word in diccionari_final_pp:
    print word[0], ": %d" % word[1]
    
print "\nLlistat de les paraules amb més ocurrències al discurs del PSOE:\n"
for word in diccionari_final_psoe:
    print word[0], ": %d" % word[1]

Llistat de les paraules amb més ocurrències al discurs del PP:

presupuestos : 60
año : 43
españa : 38
gobierno : 32
empleo : 32
crecimiento : 30
años : 29
económica : 28
pública : 24
país : 23
millones : 23
crisis : 21
económico : 21
política : 20
ahora : 19
creación : 18
públicas : 18
euros : 17
hoy : 16
hacer : 16

Llistat de les paraules amb més ocurrències al discurs del PSOE:

señor : 53
ustedes : 45
montoro : 41
política : 28
usted : 23
españa : 23
gobierno : 20
país : 17
años : 16
económica : 14
millones : 14
problema : 14
presupuestos : 12
rajoy : 12
desigualdad : 10
social : 10
crisis : 10
familias : 10
fiscal : 10
menos : 9


### Exportar un fitxer json amb els resultats obtinguts 

In [14]:
import json

# PP
wordCounterJson_pp = json.dumps(diccionari_final_pp)
print wordCounterJson_pp

with open('diccionari_final_pp.json', 'w') as outfile:
    json.dump(diccionari_final_pp, outfile)
    
# PSOE
wordCounterJson_psoe = json.dumps(diccionari_final_psoe)
print wordCounterJson_psoe

with open('diccionari_final_psoe.json', 'w') as outfile:
    json.dump(diccionari_final_psoe, outfile)

[["presupuestos", 60], ["a\u00f1o", 43], ["espa\u00f1a", 38], ["gobierno", 32], ["empleo", 32], ["crecimiento", 30], ["a\u00f1os", 29], ["econ\u00f3mica", 28], ["p\u00fablica", 24], ["pa\u00eds", 23], ["millones", 23], ["crisis", 21], ["econ\u00f3mico", 21], ["pol\u00edtica", 20], ["ahora", 19], ["creaci\u00f3n", 18], ["p\u00fablicas", 18], ["euros", 17], ["hoy", 16], ["hacer", 16]]
[["se\u00f1or", 53], ["ustedes", 45], ["montoro", 41], ["pol\u00edtica", 28], ["usted", 23], ["espa\u00f1a", 23], ["gobierno", 20], ["pa\u00eds", 17], ["a\u00f1os", 16], ["econ\u00f3mica", 14], ["millones", 14], ["problema", 14], ["presupuestos", 12], ["rajoy", 12], ["desigualdad", 10], ["social", 10], ["crisis", 10], ["familias", 10], ["fiscal", 10], ["menos", 9]]


## Representació dels resultats amb D3

- Bar chart que carrega un fitxer json (a partir d'un exemple de la classe de visualització)

In [15]:
%%html
<style>
.pp_chart div {
  font: 10px sans-serif;
  background-color: steelblue;
  text-align: right;
  padding: 3px;
  margin: 1px;
  color: white;
}

.psoe_chart div {
  font: 10px sans-serif;
  background-color: red;
  text-align: right;
  padding: 3px;
  margin: 1px;
  color: white;
}
</style>

<div id="my_pp_chart" class="pp_chart"></div>
<div id="my_psoe_chart" class="psoe_chart"></div>

In [16]:
%%javascript
    // We load the d3.js library from the Web.
    require.config({paths: {d3: "http://d3js.org/d3.v3.min"}});
    require(["d3"], function(d3) {
    
    var pp_div = d3.select("#my_pp_chart");
        
    d3.json("diccionari_final_pp.json", function(data) {
        
        pp_div.selectAll("div")
            .data(data)
            .enter().append("div")
            .style("width", function(d) { return d[1]*10 + "px"; })
            .text(function(d) { return d[0] + " (" + d[1] + ")"; });
    });
        
    var psoe_div = d3.select("#my_psoe_chart");
        
    d3.json("diccionari_final_psoe.json", function(data) {
        
        psoe_div.selectAll("div")
            .data(data)
            .enter().append("div")
            .style("width", function(d) { return d[1]*10 + "px"; })
            .text(function(d) { return d[0] + " (" + d[1] + ")"; });
    });
});

- Un altre exemple de Bar Chart ( a partir de https://gist.github.com/tristanwietsma/6047126 )

    [ En construcció     :-)     ]

In [17]:
%%html
<style>
    text {
        font: 8px; sans-serif;
    }

    .bar { fill: steelblue; }
</style>

<div id="my_chart2"></div>

In [18]:
%%javascript
    // We load the d3.js library from the Web.
    require.config({paths: {d3: "http://d3js.org/d3.v3.min"}});
    require(["d3"], function(d3) {
 
        var barChartDataSet = [];
        
        var w = 550,
        h = 900;        
        
        var svg = d3.select("#my_chart2")
            .append("svg")
            .attr("width", w*2)
            .attr("height", h / 2);
    
        d3.json("diccionari_final_pp.json", function(data) {
            processData(data, "PP", barChartDataSet);
        });
        
        d3.json("diccionari_final_psoe.json", function(data) {
            processData(data, "PSOE", barChartDataSet);
            barChart(barChartDataSet);
        });
        
        function barChart(data) {            
    
            var max_n = 0;
            for (var i = 0; i < data.length; i++) {
                max_n = Math.max(data[i][1], max_n);
            }
        
            var dx = w / max_n;
            var sp = 5;
            var dy = h / data.length - sp;
    
            // bars
            var bars = svg.selectAll(".bar")
                .data(data)
                .enter()
                .append("rect")
                .attr("class", function(d, i) {return "bar" /* "bar " + d[0]; */ })
                .attr("x", function(d, i) { if (d[2]=="PP") return 0; else return w;})
                .attr("y", function(d, i) { if (d[2]=="PP") return (dy+sp)*i; else return (dy+sp)*(i-20); })
                .attr("width", function(d, i) {return dx*d[1];})
                .attr("height", dy)
                .style("fill", function(d,i) { if (d[2]=="PP") return "steelblue"; else return "red" });
    
            // labels
            var text = svg.selectAll("text")
                .data(data)
                .enter()
                .append("text")
                .attr("class", function(d, i) {return "label " + d[0];})
                .attr("x", function(d, i) { if (d[2]=="PP") return sp; else return sp+w;})
                .attr("y", function(d, i) { if (d[2]=="PP") return (dy+sp)*i+(2/3)*dy; else return (dy+sp)*(i-20)+(2/3)*dy;})
                .attr("font-size", dy + "px")
                .style("font-weight", "bold")
                .text( function(d) {return d[0] + " (" + (100*d[1]).toFixed(2)  + "%)";});
        }
        
    function processData(data, package_name, barChartDataSet) { 
        
        var total = 0;
        for (var i = 0; i < data.length; i++) {
            total = total + data[i][1];
        }

        for (var i = 0; i < data.length; i++) { 
            barChartDataSet.push([data[i][0], data[i][1]/total, package_name]);
        }
    }        
});

- Exemple de Bubble Chart ( extret amb modificacions de: http://bl.ocks.org/mbostock/4063269 )

In [19]:
%%html
<style>
    text {
        font: sans-serif;
    }
</style>

<div id="my_first_bubble_chart"></div>

In [20]:
%%javascript
    // We load the d3.js library from the Web.
    require.config({paths: {d3: "http://d3js.org/d3.v3.min"}});
    require(["d3"], function(d3) {

    var bubbleChartDataSet = [];

    var diameter = 960,
        // https://github.com/mbostock/d3/wiki/Ordinal-Scales#categorical-colors
        color = d3.scale.category20c();

    var bubble = d3.layout.pack()
        .sort("ascending") /* "ascending", "descending", null */
        .size([diameter, diameter])
        .padding(1.5);

    var svg = d3.select("#my_first_bubble_chart").append("svg")
        .attr("width", diameter)
        .attr("height", diameter)
        .attr("class", "bubble");
        
    d3.json("diccionari_final_pp.json", function(data) {
        processData(data, "PP", bubbleChartDataSet);
    });
        
    d3.json("diccionari_final_psoe.json", function(data) {
        processData(data, "PSOE", bubbleChartDataSet);
        bubbleChart(bubbleChartDataSet);
    });
        
    function bubbleChart(dataSet) {
        var node = svg.selectAll(".node")
          .data(bubble.nodes({children: dataSet})
          .filter(function(d) { return !d.children; }))
        .enter().append("g")
          .attr("class", "node")
          .attr("transform", function(d) { return "translate(" + d.x + "," + d.y + ")"; });

        node.append("title")
          .text(function(d) { return d.className + ": " + "(" + (100*d.value).toFixed(2) + "%)"; });

        node.append("circle")
          .attr("r", function(d) { return d.r; })
          .style("fill", function(d) { if (d.packageName=="PP") return "steelblue"; else return "red" });

        node.append("text")
          .attr("dy", ".3em")
          .style("text-anchor", "middle")
          .style("font-size", function(d) { return Math.round(d.r/3)+"px"; })
          .text(function(d) { return d.className; });
    }
        
    function processData(data, package_name, bubbleChartDataSet) { 
        
        var total = 0;
        for (var i = 0; i < data.length; i++) {
            total = total + data[i][1];
        }

        for (var i = 0; i < data.length; i++) { 
            bubbleChartDataSet.push({packageName: package_name, className: data[i][0], value: data[i][1]/total});
        }
    }

    d3.select(self.frameElement).style("height", diameter + "px");
});